In [1]:
# this ensures that all C output is redirected to the Jupyter output box
%load_ext wurlitzer
import IPython.display

from xspec import *

##Multiple Models: a Background Modeling Example

We now demonstrate how to fit multiple models, each with their own response, to the same dataset. There are several reasons why this may be useful, for instance:

    We are using data from a coded aperture mask. If there are multiple sources in the field they will all contribute to the spectrum from each detector. However, each source may have a different response due to its position.
    We are observing an extended source using a telescope whose PSF is large enough that the signal from different regions are mixed together. In this case we will want to analyze spectra from all regions of the source simultaneously with each spectrum having a contribution from the model in other regions.
    We wish to model the background spectrum that includes a particle component. The particle background will have a different response from the X-ray background because the particles come from all directions, not just down the telescope.

We will demonstrate the third example here. Suppose we have a model for the background spectrum that requires a different response to that for the source spectrum. The spectra and corresponding response matrices were obtained from the HEASARC and are available from https://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/walkthrough.tar.gz

Read in the source and background spectra as separate files. 


In [2]:
%cd data
AllData("1:1 hisum.pha 2:2 losum.pha")
s1 = AllData(1)
s2 = AllData(2)
s1.response = "ginga_lac.rsp"
s2.response = "ginga_lac.rsp"

/Users/karnaud/Jupyter-notebooks/data

2 spectra  in use
 
Spectral Data File: hisum.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  1.371e+03 +/- 3.123e+00
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-48
  Telescope: GINGA Instrument: LAC  Channel Type: PHA
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            ginga_lac.rsp for Source 1

Spectral Data File: losum.pha  Spectrum 2
Net count rate (cts/s) for Spectrum:2  1.401e+02 +/- 3.549e-01
 Assigned to Data Group 2 and Plot Group 2
  Noticed Channels:  1-48
  Telescope: GINGA Instrument: LAC  Channel Type: PHA
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            ginga_lac.rsp for Source 1

Response successfully loaded.
Response successfully loaded.


Set up the model for the source. Here we will take the simple case of an absorbed power-law and set the model normalization to zero for the background spectrum. 

In [3]:
Model("phabs * pow")
m1 = AllModels(1)
m2 = AllModels(2)
m2(3).values = "0.0,-1.0"


Model phabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
                           Data group: 1
   1    1   phabs      nH         10^22    1.00000      +/-  0.0          
   2    2   powerlaw   PhoIndex            1.00000      +/-  0.0          
   3    2   powerlaw   norm                1.00000      +/-  0.0          
                           Data group: 2
   4    1   phabs      nH         10^22    1.00000      = p1
   5    2   powerlaw   PhoIndex            1.00000      = p2
   6    2   powerlaw   norm                1.00000      = p3
________________________________________________________________________


Fit statistic  : Chi-Squared              2.214313e+07     using 48 bins.
                 Chi-Squared              3.817637e+08     using 48 bins.
Total fit statistic                       4.039068e+08     with 93 d.o.f.

Test statistic : Chi-Squared              4.039068e+08     using 96 bins.
 Null hypothesis probabi

Now we need to set up the background model for both datasets with the appropriate response matrices. 

In [4]:
s1.multiresponse[1] = "ginga_lac.rsp"
s2.multiresponse[1] = "ginga_lac.rsp"

Response successfully loaded.

Fit statistic  : Chi-Squared              2.214313e+07     using 48 bins.
                 Chi-Squared                177171.1     using 48 bins.
Total fit statistic                       2.232030e+07     with 93 d.o.f.

Test statistic : Chi-Squared              2.232030e+07     using 96 bins.
 Null hypothesis probability of 0.000000e+00 with 93 degrees of freedom
 Current data and model not fit yet.
Response successfully loaded.

Fit statistic  : Chi-Squared              2.214313e+07     using 48 bins.
                 Chi-Squared                177171.1     using 48 bins.
Total fit statistic                       2.232030e+07     with 93 d.o.f.

Test statistic : Chi-Squared              2.232030e+07     using 96 bins.
 Null hypothesis probability of 0.000000e+00 with 93 degrees of freedom
 Current data and model not fit yet.


This tells XSPEC that both these datasets have a second model which must be multiplied by the appropriate response matrix for its contribution to the source region and for its contribution to the background region. This is likely to be the case for a standard imaging data where the response will only depend on the extraction region. Note that for a coded-aperture mask the situation may be more complicated with a different response for the source and the background even if they are extracted from the same region of the detector. We now define the background model to be used. In this case take the simple example of a single power-law 

In [5]:
m1_2 = Model("pow","myback",2)


Model myback:powerlaw<1> Source No.: 2   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
                           Data group: 1
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0          
                           Data group: 2
   3    1   powerlaw   PhoIndex            1.00000      = myback:p1
   4    1   powerlaw   norm                1.00000      = myback:p2
________________________________________________________________________


Fit statistic  : Chi-Squared              9.779387e+07     using 48 bins.
                 Chi-Squared              4.523427e+08     using 48 bins.
Total fit statistic                       5.501366e+08     with 91 d.o.f.

Test statistic : Chi-Squared              5.501366e+08     using 96 bins.
 Null hypothesis probability of 0.000000e+00 with 91 degrees of freedom
 Current data and model not fit yet.


We have now set up XSPEC so that the source data is compared to a source model multiplied by the source response plus a background model multiplied by the background response and the background data is compared to the background model multiplied by the background response. The background models fitted to the source and background data are constrained to be the same. 

In [6]:
AllModels.show()


Parameters defined:
Model phabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
                           Data group: 1
   1    1   phabs      nH         10^22    1.00000      +/-  0.0          
   2    2   powerlaw   PhoIndex            1.00000      +/-  0.0          
   3    2   powerlaw   norm                1.00000      +/-  0.0          
                           Data group: 2
   4    1   phabs      nH         10^22    1.00000      = p1
   5    2   powerlaw   PhoIndex            1.00000      = p2
   6    2   powerlaw   norm                0.0          frozen
________________________________________________________________________

Model myback:powerlaw<1> Source No.: 2   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
                           Data group: 1
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0         